<a href="https://colab.research.google.com/github/AbdullahAlTalaq/AbdullahAlTalaq/blob/main/Pan_journal_paper_(A)3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Importing necessary libraries for machine learning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
# classifiers
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Importing imbalanced learning libraries
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN, BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE, SMOTEN, SMOTENC, KMeansSMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, OneSidedSelection, NeighbourhoodCleaningRule
from imblearn.under_sampling import CondensedNearestNeighbour, ClusterCentroids, NearMiss
from imblearn.under_sampling import OneSidedSelection, EditedNearestNeighbours, RepeatedEditedNearestNeighbours
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
# Scaling technique
scaler = StandardScaler()

# Oversampling techniques (8)
smote = SMOTE()
ros = RandomOverSampler()
adasyn = ADASYN()
borderlinessmote = BorderlineSMOTE()
svmsmote = SVMSMOTE()
smoten = SMOTEN()
smotenc = SMOTENC(categorical_features=False)
kmeanssmote = KMeansSMOTE()

# Undersampling techniques (10)
randomundersampling = RandomUnderSampler()
tomeklinks = TomekLinks()
onesidedselection = OneSidedSelection()
neighbourhoddcleaningrule = NeighbourhoodCleaningRule()
condensednearestneighbour = CondensedNearestNeighbour()
clustercentroids = ClusterCentroids()
nearmiss = NearMiss()
onesidedselection = OneSidedSelection()
editednearestneighbours = EditedNearestNeighbours()
repeatededitednearestneighbours = RepeatedEditedNearestNeighbours()


In [72]:
def stratified_cv(classifier, X_train, y_train):
    skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for train_index, test_index in skf.split(X_train, y_train):
        X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

        classifier.fit(X_train_fold, y_train_fold)
        y_pred = classifier.predict(X_test_fold)

        accuracy_scores.append(accuracy_score(y_test_fold, y_pred))
        precision_scores.append(precision_score(y_test_fold, y_pred, average='weighted'))
        recall_scores.append(recall_score(y_test_fold, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test_fold, y_pred, average='weighted'))

    print(f'Average Accuracy: {sum(accuracy_scores) / 5}')
    print(f'Average Precision: {sum(precision_scores) / 5}')
    print(f'Average Recall: {sum(recall_scores) / 5}')
    print(f'Average F1 Score: {sum(f1_scores) / 5}')

def grid_search_and_fit(classifier, param_grid, X, y, verbosity=2):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

    # Print results before tuning
    print("Model results with default hyperparameters")
    stratified_cv(classifier, X_train, y_train)

    # Perform grid search
    grid_search = GridSearchCV(classifier, param_grid, cv=3, verbose=verbosity, scoring="f1_weighted")
    grid_search.fit(X_train, y_train)

    # Get the best parameters
    best_params = grid_search.best_params_

    # Fit the classifier with the best parameters
    classifier.set_params(**best_params)
    classifier.fit(X_train, y_train)

    # Print results after hyperparameter tuning
    print("\n\nModel results after hyperparameter tuning")
    stratified_cv(classifier, X_train, y_train)

In [28]:
# Define and test grid_search_and_fit function
def test_grid_search_and_fit(classifier, param_grid, X, y):
    grid_search_and_fit(classifier, param_grid, X, y, verbosity=2)

In [29]:
df =pd.read_csv('/content/Debernardi et al 2020 data.csv')

In [30]:
df = df.drop(['sample_id', 'patient_cohort', 'sample_origin'], axis=1)

In [31]:
df['sex'] = df['sex'].map({'M': 1, 'F': 0})

In [32]:
df.loc[df['diagnosis'].isin([3, 2]), 'diagnosis'] = 0

In [33]:
stage_mapping = {
    'I': 1,
    'IA': 2,
    'IB': 3,
    'II': 4,
    'IIA': 5,
    'IIB': 6,
    'III': 7,
    'IV': 8
}

df['stage'] = df['stage'].map(stage_mapping)

In [34]:
df_mean = df.fillna(df.mean())

<ipython-input-34-19360f3909b6>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_mean = df.fillna(df.mean())


In [35]:
df_mean = df_mean.drop(['benign_sample_diagnosis'], axis=1)

In [36]:
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
df_knn_imputed=df.copy()
imputer = SimpleImputer(strategy='mean')
df_knn_imputed[['REG1A', 'plasma_CA19_9']] = imputer.fit_transform(df[['REG1A', 'plasma_CA19_9']])
features = ['age', 'sex', 'diagnosis', 'plasma_CA19_9', 'creatinine', 'LYVE1', 'REG1B', 'TFF1', 'REG1A']
knn_imputer = KNNImputer(n_neighbors=10)
missing_stage_index = df_knn_imputed[df_knn_imputed['stage'].isnull()].index

imputed_values = knn_imputer.fit_transform(df[features + ['stage']])
df_knn_imputed.loc[missing_stage_index, 'stage'] = imputed_values[missing_stage_index, -1]

In [37]:
df_knn_imputed = df_knn_imputed.drop(['benign_sample_diagnosis'], axis=1)

In [38]:
df_mean['stage'] = df_mean['stage'].astype('int64')
df_knn_imputed['stage']=df_knn_imputed['stage'].astype('int64')


In [39]:
df_mean['diagnosis'].unique()

array([1, 0])

#df_mean

In [50]:
X = df_mean.drop(columns=['diagnosis', 'stage', 'plasma_CA19_9'])
y = df_mean['diagnosis']  # Target variable



# Define the parameter grid for DecisionTreeClassifier
dt_param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3],
    'max_features': [None, 'sqrt', 'log2'],
    'random_state': [42]
}
test_grid_search_and_fit(
    classifier=DecisionTreeClassifier(),
    param_grid=dt_param_grid,
    X=X,
    y=y
)

Model results with default hyperparameters
Average Accuracy: 0.8947650663942799
Average Precision: 0.8934590937155826
Average Recall: 0.8947650663942799
Average F1 Score: 0.8900687313113901
Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42, splitter=random; t

In [80]:
X = df_mean.drop(columns=['diagnosis', 'stage', 'plasma_CA19_9'])
y = df_mean['diagnosis']  # Target variable



# Define the parameter grid for RF
nb_param_grid = {
    'priors': [None, [0.3, 0.7], [0.4, 0.6]]  # Prior probabilities of the classes
}

test_grid_search_and_fit(
    classifier=GaussianNB(),#KNN
    param_grid=nb_param_grid,
    X=X,
    y=y
)

Model results with default hyperparameters
Average Accuracy: 0.7525280898876404
Average Precision: 0.9267782986542553
Average Recall: 0.7525280898876404
Average F1 Score: 0.7615133362696211
Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END ........................................priors=None; total time=   0.0s
[CV] END ........................................priors=None; total time=   0.0s
[CV] END ........................................priors=None; total time=   0.0s
[CV] END ..................................priors=[0.3, 0.7]; total time=   0.0s
[CV] END ..................................priors=[0.3, 0.7]; total time=   0.0s
[CV] END ..................................priors=[0.3, 0.7]; total time=   0.0s
[CV] END ..................................priors=[0.4, 0.6]; total time=   0.0s
[CV] END ..................................priors=[0.4, 0.6]; total time=   0.0s
[CV] END ..................................priors=[0.4, 0.6]; total time=   0.0s


Model results after 

In [79]:
X = df_mean.drop(columns=['diagnosis', 'stage', 'plasma_CA19_9'])
y = df_mean['diagnosis']  # Target variable



# Define the parameter grid for KNN
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9],          # Number of neighbors to consider
    'weights': ['uniform', 'distance'],   # Weight function used in predictions
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute neighbors
    'leaf_size': [10, 20, 30],            # Leaf size for tree-based algorithms
    'p': [1, 2]                           # Power parameter for the Minkowski metric (1 for Manhattan, 2 for Euclidean)
}
test_grid_search_and_fit(
    classifier=KNeighborsClassifier(),#KNN
    param_grid=knn_param_grid,
    X=X,
    y=y
)

Model results with default hyperparameters
Average Accuracy: 0.8792134831460675
Average Precision: 0.876658631167588
Average Recall: 0.8792134831460675
Average F1 Score: 0.8763994586599463
Fitting 3 folds for each of 192 candidates, totalling 576 fits
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=1, weights=uniform; total time=   0.1s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=1, weights=uniform; total time=   0.1s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=1, weights=uniform; total time=   0.1s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=1, weights=distance; total time=   0.0s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=1, weights=distance; total time=   0.0s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=1, weights=distance; total time=   0.0s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=2, weights=uniform; total time=   0.0s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=2, weights=uniform; tota

KeyboardInterrupt: 

In [78]:
X = df_mean.drop(columns=['diagnosis', 'stage', 'plasma_CA19_9'])
y = df_mean['diagnosis']  # Target variable



# Define the parameter grid for RF
rf_param_grid = {
    'n_estimators': [50, 100],           # Number of trees in the forest
    'criterion': ['gini', 'entropy'],             # Splitting criterion
    'max_depth': [None, 5, 10, 15],                # Maximum depth of the trees
    'min_samples_split': [2, 5, 10, 20],           # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 3, 5],              # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2'],      # Number of features to consider when looking for the best split
    'bootstrap': [True, False],                    # Whether to use bootstrapping
    'random_state': [42]                           # Random seed for reproducibility
}
test_grid_search_and_fit(
    classifier=RandomForestClassifier(),#RF
    param_grid=rf_param_grid,
    X=X,
    y=y
)

Model results with default hyperparameters
Average Accuracy: 0.9717313585291114
Average Precision: 0.9680470319887668
Average Recall: 0.9717313585291114
Average F1 Score: 0.9604462746379344
Fitting 3 folds for each of 1536 candidates, totalling 4608 fits
[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50, random_state=42; total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50, random_state=42; total time=   0.1s
[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50, random_state=42; total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100, random_state=42; total time=   0.2s
[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100, random_state=42; total time=   0.2s


...

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=50, random_state=42; total time=   0.1s
[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=50, random_state=42; total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=50, random_state=42; total time=   0.1s


KeyboardInterrupt: 

In [53]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Base classifier
base_classifier = DecisionTreeClassifier(random_state=42)

X = df_mean.drop(columns=['diagnosis', 'stage', 'plasma_CA19_9'])
y = df_mean['diagnosis']  # Target variable



# Bagging Classifier Parameter Grid
bagging_param_grid = {
    'base_estimator': [base_classifier],
    'n_estimators': [50, 100, 150],         # Number of base classifiers in the ensemble
    'max_samples': [0.5, 0.7, 0.9],        # Fraction of samples used for training each base classifier
    'max_features': [0.5, 0.7, 0.9],       # Fraction of features used for training each base classifier
    'random_state': [42]
}

test_grid_search_and_fit(
    classifier=BaggingClassifier(),#KNN
    param_grid=bagging_param_grid,
    X=X,
    y=y
)

Model results with default hyperparameters
Average Accuracy: 0.9400153217568947
Average Precision: 0.9336714470131537
Average Recall: 0.9400153217568947
Average F1 Score: 0.9248163833875159
Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END base_estimator=DecisionTreeClassifier(random_state=42), max_features=0.5, max_samples=0.5, n_estimators=50, random_state=42; total time=   0.2s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


[CV] END base_estimator=DecisionTreeClassifier(random_state=42), max_features=0.5, max_samples=0.5, n_estimators=50, random_state=42; total time=   0.2s
[CV] END base_estimator=DecisionTreeClassifier(random_state=42), max_features=0.5, max_samples=0.5, n_estimators=50, random_state=42; total time=   0.2s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


[CV] END base_estimator=DecisionTreeClassifier(random_state=42), max_features=0.5, max_samples=0.5, n_estimators=100, random_state=42; total time=   0.3s


...

[CV] END base_estimator=DecisionTreeClassifier(random_state=42), max_features=0.9, max_samples=0.9, n_estimators=150, random_state=42; total time=   0.4s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(




Model results after hyperparameter tuning


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en

Average Accuracy: 0.9513023493360571
Average Precision: 0.9484698235572754
Average Recall: 0.9513023493360571
Average F1 Score: 0.9426036049245216


#df_knn_imputed

In [77]:
X = df_knn_imputed.drop(columns=['diagnosis', 'stage', 'plasma_CA19_9'])
y = df_knn_imputed['diagnosis']  # Target variable



# Define the parameter grid for KNN
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9],          # Number of neighbors to consider
    'weights': ['uniform', 'distance'],   # Weight function used in predictions
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute neighbors
    'leaf_size': [10, 20, 30],            # Leaf size for tree-based algorithms
    'p': [1, 2]                           # Power parameter for the Minkowski metric (1 for Manhattan, 2 for Euclidean)
}
test_grid_search_and_fit(
    classifier=KNeighborsClassifier(),#KNN
    param_grid=knn_param_grid,
    X=X,
    y=y
)

Model results with default hyperparameters
Average Accuracy: 0.8792134831460675
Average Precision: 0.876658631167588
Average Recall: 0.8792134831460675
Average F1 Score: 0.8763994586599463
Fitting 3 folds for each of 192 candidates, totalling 576 fits
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=1, weights=uniform; total time=   0.0s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=1, weights=uniform; total time=   0.0s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=1, weights=uniform; total time=   0.0s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=1, weights=distance; total time=   0.0s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=1, weights=distance; total time=   0.0s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=1, weights=distance; total time=   0.0s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=2, weights=uniform; total time=   0.0s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=3, p=2, weights=uniform; tota

In [62]:
X = df_knn_imputed.drop(columns=['diagnosis', 'stage', 'plasma_CA19_9'])
y = df_knn_imputed['diagnosis']  # Target variable



# Define the parameter grid for RF
svm_param_grid = {
    'C': [1.0],                # Choose a single value or a smaller set
    'kernel': ['linear', 'rbf'],  # Select fewer kernels
    'gamma': ['scale'],           # Choose a single gamma value
    'random_state': [42]
}

test_grid_search_and_fit(
    classifier=SVC(),#KNN
    param_grid=svm_param_grid,
    X=X,
    y=y
)

Model results with default hyperparameters
Average Accuracy: 0.8271450459652707
Average Precision: 0.5701589710529913
Average Recall: 0.8271450459652707
Average F1 Score: 0.6750149241453064
Fitting 3 folds for each of 2 candidates, totalling 6 fits


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

KeyboardInterrupt: 

In [76]:
X = df_knn_imputed.drop(columns=['diagnosis', 'stage', 'plasma_CA19_9'])
y = df_knn_imputed['diagnosis']  # Target variable



# Define the parameter grid for RF
nb_param_grid = {
    'priors': [None, [0.3, 0.7], [0.4, 0.6]]  # Prior probabilities of the classes
}

test_grid_search_and_fit(
    classifier=GaussianNB(),#KNN
    param_grid=nb_param_grid,
    X=X,
    y=y
)

Model results with default hyperparameters
Average Accuracy: 0.7525280898876404
Average Precision: 0.9267782986542553
Average Recall: 0.7525280898876404
Average F1 Score: 0.7615133362696211
Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END ........................................priors=None; total time=   0.0s
[CV] END ........................................priors=None; total time=   0.0s
[CV] END ........................................priors=None; total time=   0.0s
[CV] END ..................................priors=[0.3, 0.7]; total time=   0.0s
[CV] END ..................................priors=[0.3, 0.7]; total time=   0.0s
[CV] END ..................................priors=[0.3, 0.7]; total time=   0.0s
[CV] END ..................................priors=[0.4, 0.6]; total time=   0.0s
[CV] END ..................................priors=[0.4, 0.6]; total time=   0.0s
[CV] END ..................................priors=[0.4, 0.6]; total time=   0.0s


Model results after 

In [75]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Base classifier
base_classifier = DecisionTreeClassifier(random_state=42)

X = df_knn_imputed.drop(columns=['diagnosis', 'stage', 'plasma_CA19_9'])
y = df_knn_imputed['diagnosis']  # Target variable



# Bagging Classifier Parameter Grid
bagging_param_grid = {
    'base_estimator': [base_classifier],
    'n_estimators': [50, 100, 150],         # Number of base classifiers in the ensemble
    'max_samples': [0.5, 0.7, 0.9],        # Fraction of samples used for training each base classifier
    'max_features': [0.5, 0.7, 0.9],       # Fraction of features used for training each base classifier
    'random_state': [42]
}

test_grid_search_and_fit(
    classifier=BaggingClassifier(),#KNN
    param_grid=bagging_param_grid,
    X=X,
    y=y
)

Model results with default hyperparameters
Average Accuracy: 0.9355209397344229
Average Precision: 0.9283536106238757
Average Recall: 0.9355209397344229
Average F1 Score: 0.921273752525796
Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END base_estimator=DecisionTreeClassifier(random_state=42), max_features=0.5, max_samples=0.5, n_estimators=50, random_state=42; total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


[CV] END base_estimator=DecisionTreeClassifier(random_state=42), max_features=0.5, max_samples=0.5, n_estimators=50, random_state=42; total time=   0.1s
[CV] END base_estimator=DecisionTreeClassifier(random_state=42), max_features=0.5, max_samples=0.5, n_estimators=50, random_state=42; total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


[CV] END base_estimator=DecisionTreeClassifier(random_state=42), max_features=0.5, max_samples=0.5, n_estimators=100, random_state=42; total time=   0.2s


...

[CV] END base_estimator=DecisionTreeClassifier(random_state=42), max_features=0.9, max_samples=0.7, n_estimators=150, random_state=42; total time=   0.6s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


[CV] END base_estimator=DecisionTreeClassifier(random_state=42), max_features=0.9, max_samples=0.7, n_estimators=150, random_state=42; total time=   0.6s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


KeyboardInterrupt: 

In [74]:
X = df_knn_imputed.drop(columns=['diagnosis', 'stage', 'plasma_CA19_9'])
  # Features
y = df_knn_imputed['diagnosis']  # Target variable



# Define the parameter grid for KNN
dt_param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3],
    'max_features': [None, 'sqrt', 'log2'],
    'random_state': [42]
}
test_grid_search_and_fit(
    classifier=DecisionTreeClassifier(),#KNN
    param_grid=dt_param_grid,
    X=X,
    y=y
)

Model results with default hyperparameters
Average Accuracy: 0.8924923391215526
Average Precision: 0.8910908928165874
Average Recall: 0.8924923391215526
Average F1 Score: 0.8878699834471362
Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42, splitter=best; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42, splitter=random; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42, splitter=random; t

In [73]:
X = df_knn_imputed.drop(columns=['diagnosis', 'stage', 'plasma_CA19_9'])  # Features
y = df_knn_imputed['diagnosis']  # Target variable



# Define the parameter grid for RF
rf_param_grid = {
    'n_estimators': [50, 100],           # Number of trees in the forest
    'criterion': ['gini', 'entropy'],             # Splitting criterion
    'max_depth': [None, 5, 10, 15],                # Maximum depth of the trees
    'min_samples_split': [2, 5, 10, 20],           # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 3, 5],              # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2'],      # Number of features to consider when looking for the best split
    'bootstrap': [True, False],                    # Whether to use bootstrapping
    'random_state': [42]                           # Random seed for reproducibility
}
test_grid_search_and_fit(
    classifier=RandomForestClassifier(),#KNN
    param_grid=rf_param_grid,
    X=X,
    y=y
)

Model results with default hyperparameters
Average Accuracy: 0.9603932584269662
Average Precision: 0.9554454185029326
Average Recall: 0.9603932584269662
Average F1 Score: 0.9494626378803013
Fitting 3 folds for each of 1536 candidates, totalling 4608 fits
[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50, random_state=42; total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50, random_state=42; total time=   0.1s
[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50, random_state=42; total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100, random_state=42; total time=   0.2s
[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100, random_state=42; total time=   0.2s


...

[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=100, random_state=42; total time=   0.3s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=100, random_state=42; total time=   0.3s
[CV] END bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=20, n_estimators=50, random_state=42; total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


KeyboardInterrupt: 